In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Contents of the Notebook:

### Part1: Exploratory Data Analysis(EDA):
1)Analysis of the features.

2)Finding any relations or trends considering multiple features.

### Part2: Feature Engineering and Data Cleaning:
1)Adding any few features.

2)Removing redundant features.

3)Converting features into suitable form for modeling.

### Part3: Predictive Modeling
1)Running Basic Algorithms.

2)Cross Validation.

3)Ensembling.

4)Important Features Extraction.

# Part1: Exploratory Data Analysis(EDA)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
data=pd.read_csv('/kaggle/input/titanic-machine-learning-from-disaster/train.csv')

In [ ]:
data.head()

In [ ]:
data.isnull().sum() #널 값 확인

Age, Cabin and Embarked에 널 값 확인, 나중에 값을 추가해야겠다.

#### How many Survived??

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,8)) #fplt.subplots : 그래프 만들기, igsize(가로길이, 세로길이)
data['Survived'].value_counts().plot.pie(explode=[0,0.1],autopct='%1.1f%%', ax=ax[0],shadow=True)
ax[0].set_title('Survived')
ax[0].set_ylabel('')
sns.countplot('Survived',data=data,ax=ax[1])
ax[1].set_title('Survived')
plt.show()

891명 중 350명 단, 38.4%만이 살아 남았다.
하지만, 이러한 정보만으로는 유의미한 데이터를 가졌다고 할 수 없다.

Age, Embarcation, Sex등 카테고리를 이용하여, 데이터를 세분화시키자.

### Types Of Features

#### Categorical Features(범주형): 
여러가지 특성으로 분류 할 수 있다.

ex)Sex(남, 여), Embakred(Southampton, Liverpool)

#### Ordinal Features(순서성):
Categorical Features와 비슷하지만, 수치화 할 수 있다는 것이 특징. '상대적이다'라는 표현을 쓸 수 있는 용이성이 있다.

ex)높이 : 높은, 중간층, 낮음, Plcass : 1등급, 2등급, 3등

#### Continous Feature(연속성):
두 개의 값 사이에 값을 가지거나, 최댓값과, 최솟값 사이에 값들이 있다.

ex) 나이(100세 0세 사이의 나이들)




# Analysing The Features

### Sex -> Categorical Feature

In [ ]:
data.groupby(['Sex', 'Survived'])['Survived'].count()

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,8))
data[['Sex','Survived']].groupby(['Sex']).mean().plot.bar(ax=ax[0])
ax[0].set_title('Survived vs Sex')
sns.countplot('Sex',hue='Survived',data=data,ax=ax[1])
ax[1].set_title('Sex:Survived vs Dead')
plt.show()

승선은 남자가 여자보다 훨씬 많이 했다. 하지만, '살아남은' 숫자는 2배 이상 차이난다.
승선한 여성이 살아남은 비율은 75%, 반면에, 남성은 18%다.

=> 데이터 분석할 때, 유의미한 자료가 될 것이다.

## Pclass -> Ordinal Feature

In [ ]:
pd.crosstab(data.Pclass, data.Survived,margins=True).style.background_gradient(cmap='summer_r')

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,8))
data['Pclass'].value_counts().plot.bar(color=['#CD7F32','#FFDF00','#D3D3D3'],ax=ax[0])
ax[0].set_title('Number Of Passengers By Pclass')
ax[0].set_ylabel('Count')
sns.countplot('Pclass', hue='Survived', data=data,ax=ax[1])
ax[1].set_title('Pclass:Survived vs Dead')
plt.show()

1등급 63%, 2등급 48%, 3등급 25% 정도만 살았다. 높은 등급일 수록 산 사람의 비율이 높아졌다.

성별과 좌석 등급을 같이 엮어서 보자

In [ ]:
pd.crosstab([data.Sex, data.Survived], data.Pclass, margins=True).style.background_gradient(cmap='summer_r')

In [ ]:
sns.factorplot('Pclass', 'Survived', hue='Sex', data=data)
plt.show()

FactorPlot why? 범주의 분리를 쉽게 나타낼 수 있다.

단서
1. 1등급 여성 94명 중 단, 3명만이 죽었다.

2. 등급에 상관없이 여성들은 많이 살아남았고, 남자들은 많이 죽었다.

### Age->Continous Feature

In [ ]:
print('Oldest Passenger was of:',data['Age'].max(),'Years')
print('Youngest Passenger was of:',data['Age'].min(),'Years')
print('Average Age on the ship:',data['Age'].mean(),'Years')

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,8))
sns.violinplot("Pclass","Age", hue="Survived", data=data,split=True,ax=ax[0])
ax[0].set_title('Pclass and Age vs Survived')
ax[0].set_yticks(range(0,110,10))
sns.violinplot("Sex","Age", hue="Survived", data=data,split=True,ax=ax[1])
ax[1].set_title('Sex and Age vs Survived')
ax[1].set_yticks(range(0,110,10))
plt.show()

단서

1. 어린이는 클래스가 높아질 수록 많다.
2. 어린이는 클래스와 상관없이, 많이 살아남았다.
3. 1등급 20세~50세 사람들이 가장 많이 살아남았다. 그 중, 여성들의 비율이 압도적이다.
4. 남성들은 나이가 높을 수록, 살아남은 비율이 감소했다.

*나이에 널 값을 채워보자*
how ? 전체 나이를 평균으로 구하는 것은 양질의 데이터가 될 수가 없다. 그래서, '나이' 카테고리를 이용해보자.

In [ ]:
data['Initial']=0
for i in data:
    data['Initial']=data.Name.str.extract('([A-Za-z]+)\.')

In [ ]:
pd.crosstab(data.Initial,data.Sex).T.style.background_gradient(cmap='summer_r')

'Mr' = 남성
'Mrs', 'Ms' = 여성 and 'Mrs' = 결혼한 여성, 'Ms' = 결혼하지 않는 여성

이와 같이, 카테고리를 세분화할 수록 널값에 들어갈 값의 정확성이 높아진다.

In [ ]:
data['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr'],inplace=True)

In [ ]:
data.groupby('Initial')['Age'].mean()

In [ ]:
### Filling Nan Ages

In [ ]:
data.loc[(data.Age.isnull())&(data.Initial=='Mr'),'Age']=33
data.loc[(data.Age.isnull())&(data.Initial=='Mrs'),'Age']=36
data.loc[(data.Age.isnull())&(data.Initial=='Master'),'Age']=5
data.loc[(data.Age.isnull())&(data.Initial=='Miss'),'Age']=22
data.loc[(data.Age.isnull())&(data.Initial=='Other'),'Age']=46

In [ ]:
data.Age.isnull().any()

In [ ]:
f,ax=plt.subplots(1,2,figsize=(20,10))
data[data['Survived']==0].Age.plot.hist(ax=ax[0],bins=20,edgecolor='black',color='red')
ax[0].set_title('Survived= 0')
x1=list(range(0,85,5))
ax[0].set_xticks(x1)
data[data['Survived']==1].Age.plot.hist(ax=ax[1],color='green',bins=20,edgecolor='black')
ax[1].set_title('Survived= 1')
x2=list(range(0,85,5))
ax[1].set_xticks(x2)
plt.show()

단서
1. 5세 미만 아동은 크게 많이 살아남았다. (위급 사항시, '여성과 아동부터 살리자'라는 정책의 영향을 받았을 거라는 추측)

2. 가장 나이 많은 사람도 살아남았다.

3. 30대에서 40대가 가장 많이 죽었다.

In [ ]:
sns.factorplot('Pclass','Survived',col='Initial',data=data)
plt.show()

클래스와 상관없이 여성과 아이는 많이 살아남았다.

why ? 여성과 아이부터 살리자는 정첵

### Embarked -> Categorical Value

In [ ]:
pd.crosstab([data.Embarked,data.Pclass], [data.Sex,data.Survived],margins=True).style.background_gradient(cmap='summer_r')

### Chances for Survival by Port Of Embarkation

In [ ]:
sns.factorplot('Embarked','Survived',data=data)
fig=plt.gcf()
fig.set_size_inches(5,3)
plt.show()

C항구에서 승선한 사람들이 가장 많이 살아남았고, S항구에 승선한 사람들이 가장 적게 살아남았다.

In [ ]:
f,ax=plt.subplots(2,2,figsize=(20,15))
sns.countplot('Embarked', data=data, ax=ax[0,0])
ax[0,0].set_title('No. Of Passengers Boarded')
sns.countplot('Embarked', hue='Sex',data=data,ax=ax[0,1])
ax[0,1].set_title('Male-Female Split for Embarked')
sns.countplot('Embarked', hue='Survived', data=data,ax=ax[1,0])
ax[1,0].set_title('Embarked vs Survived')
sns.countplot('Embarked', hue='Pclass',data=data,ax=ax[1,1])
ax[1,1].set_title('Embarked vs Pclass')
plt.subplots_adjust(wspace=0.2,hspace=0.5)
plt.show()

단서

1. S항구에서 가장 많은 사람이 승선했다. 그리고 그들의 대부분은 3등급이었다.

2. C항구에서 승선한 사람들은 가장 많은 비율로 살아남았다. 추측하자면, 1등급과 2등급에 있었을 가능성이 높았기 때문이라고 추론한다.

3. S항구는 부유한 사람들이 대부분이었을 것이다. 하지만, 3등급에 속한 사람들 또한 많았기 때문에 죽은 비율도 꽤 높다.

4. Q항구 승선한 95% 사람들은 3등급이었다.

In [ ]:
sns.factorplot('Pclass', 'Survived', hue='Sex', col='Embarked', data=data)
plt.show()

단서

1. 1등급과 2등급 여성들은 거의 다 살아 남았다.

2. S항구에서 승선한 3등급 사람들은 남자와 여자 상관없이 많이 죽었다. (돈 문제)

3. Q항구에서 승선한 남자들이 가장 운이 없다. 그들이 가장 많이 3등급에 탑승했기 때문이다.

### Filling Embarked NaN
S항구에서 승선한 사람들이 가장 많기 때문에, "Embkared'카테고리는 'S'로 대체할 것이다.

In [ ]:
data['Embarked'].fillna('S', inplace=True)

In [ ]:
data.Embarked.isnull().any()

### SibSip -> Discrete Feature
혼자 왔는 지, 동승했는 지 알 수 있는 지표

Sibling = broter, sister, stepbrother, stepsister

Spouse = husband, wife

In [ ]:
pd.crosstab([data.SibSp],data.Survived).style.background_gradient(cmap='summer_r')

In [ ]:
f,ax=plt.subplots(1,2,figsize=(20,8))
sns.barplot('SibSp','Survived',data=data,ax=ax[0])
ax[0].set_title('SibSp vs Survived')
sns.pointplot('SibSp','Survived',data=data,ax=ax[1]) #factorplot는 그래프가 안 나옴
ax[1].set_title('SibSp vs Survived')
plt.close(2)
plt.show()

In [ ]:
pd.crosstab(data.SibSp,data.Pclass).style.background_gradient(cmap='summer_r')

단서

Siblings 없이 혼자 탄 승객은 34.5% 확률로 살아남았다. 하지만, 그 수가 증가할 수록 생존률은 감소했다. 

why? 가족을 구하기 위해, 희생할테니깐.
하지만, 놀랍게도 5-8명이 같이 탄 가족들은 한 명도 살아남지 못했다. 클래스의 차이 때문일까?

정답이다
SibSp>3은 모두 3등급 클래스다.

# Parch

In [ ]:
pd.crosstab(data.Parch,data.Pclass).style.background_gradient(cmap='summer_r')

가족 수가 많을 수록 3등급에 속해있다는 것을 보여준다.

In [ ]:
f,ax=plt.subplots(1,2,figsize=(20,8))
sns.barplot('Parch','Survived',data=data,ax=ax[0])
ax[0].set_title('Parch vs Survived')
sns.pointplot('Parch','Survived',data=data,ax=ax[1])
ax[1].set_title('Parch vs Survived')
plt.close(2)
plt.show()

단서

1. 가족과 같이 탄 승객들은 생존 비율이 높다. 단, 가족 수가 많을 수록 비율은 줄어들었다.

2. 1-3 부모와 같이 탄 사람들은 많이 살아 남았다.

3. 혼자 탑승한 것은 치명적이고, 4명 이상의 가족 또한 마찬가지다.

# Fare -> Continous Feature

In [ ]:
print('Highest Fare was:',data['Fare'].max())
print('Lowest Fare was:',data['Fare'].min())
print('Average Fare was:',data['Fare'].mean())

In [ ]:
f,ax=plt.subplots(1,3,figsize=(20,8))
sns.distplot(data[data['Pclass']==1].Fare,ax=ax[0])
ax[0].set_title('Fares in Pclass 1')
sns.distplot(data[data['Pclass']==2].Fare,ax=ax[1])
ax[1].set_title('Fares in Pclass 2')
sns.distplot(data[data['Pclass']==3].Fare,ax=ax[2])
ax[2].set_title('Fares in Pclass 3')
plt.show()

같은 등급이어도 표의 가격차이가 있다. 1등급은 가격차이가 제일 많다. 하지만, 가격은 연속적이기 때문에, 유의미한 데이터로 만들 수 있다. 

이제, binning을 해보자 !

# Observations in a Nutshell for all features:

sex : 남성보다 여성이 높은 비율로 살아남았다.

Pclass : 1등급 사람들이 가장 많이 살아남았고, 3등급은 이와 반대된다. 그리고, 1등급, 2등급 여성들은 등급을 무시하고, 제일 많이 살아남았다.

Age : 5-10 사이의 아동들이 가장 많이 살아남았으며, 15-35 사이의 사람들이 많이 죽었다.

Embarked : S항구에서 승선한 대부분 1등급 사람들 보다 C항구에서 승선한 사람들이 더 많이 살아남았따. Q항구 승객 대부분은 3등급이다.

Parch+SibSp : 1-2형제, 1-3와 같이 탄 사람들은 혼자 또는 이 보다 많이 동승한 사람들 보다 많이 살아남았다.

# Correlation Between The Features

In [ ]:
sns.heatmap(data.corr(),annot=True,cmap='RdYlGn',linewidths=0.2) #data.corr()-->correlation matrix
fig=plt.gcf()
fig.set_size_inches(10,8)
plt.show()

POSITIVE CORRELATION : B특성에서 A특성이 높아질 때, 그들은 긍정적으로 상응한다.

NEGATIVE CORRELATION: B특성에서 A특성이 낮아질 때, 그들은 부정적으로 상응한다.

# Part2: Feature Engineering and Data Cleaning

필요한 정보들은 이용하고, 불필요한 정보는 삭제하자

### Age_band

continous -> categorical values

why? 유의미한 정보를 얻기 위해서

ex) 남자, 여자 1,0과 같은
    나이 0~10 : 0, 10 ~20 : 1, 20~30 : 2와     같이 만들어 줄려고

In [ ]:
data['Age_band']=0
data.loc[data['Age']<=16,'Age_band']=0
data.loc[(data['Age']>16)&(data['Age']<=32),'Age_band']=1
data.loc[(data['Age']>32)&(data['Age']<=48),'Age_band']=2
data.loc[(data['Age']>48)&(data['Age']<=64),'Age_band']=3
data.loc[data['Age']>64,'Age_band']=4
data.head(2)

In [ ]:
data['Age_band'].value_counts().to_frame().style.background_gradient(cmap='summer')

In [ ]:
sns.factorplot('Age_band','Survived',data=data,col='Pclass')
plt.show()

등급과 상관없이 나이가 많을 수록 살아남는 확률이 줄어들었다.

# Family_Size and Alone

In [ ]:
data['Family_Size']=0
data['Family_Size']=data['Parch']+data['SibSp']#family size
data['Alone']=0
data.loc[data.Family_Size==0,'Alone']=1#Alone

f,ax=plt.subplots(1,2,figsize=(18,6))
sns.pointplot('Family_Size','Survived',data=data,ax=ax[0])
ax[0].set_title('Family_Size vs Survived')
sns.pointplot('Alone','Survived',data=data,ax=ax[1])
ax[1].set_title('Alone vs Survived')
plt.close(2)
plt.close(3)
plt.show()

혼자이거나 Familt_Size가 0이면 살아남은 확률이 굉장히 낮다. 또한, 4인 이상 가족도 마찬가지다.

In [ ]:
sns.factorplot('Alone','Survived',data=data,hue='Sex',col='Pclass')
plt.show()

성별과 클래스를 제외하고도 혼자 있는 것은 위험했다. 
3등급에 혼자 탑승한 여성은 가족과 동승한 사람들보다 많이 죽었다.

# Fare_Range

continous feature -> ordinal value

In [ ]:
data['Fare_Range']=pd.qcut(data['Fare'],4)
data.groupby(['Fare_Range'])['Survived'].mean().to_frame().style.background_gradient(cmap='summer_r')

금액이 비쌀수록, 살아남은 확률도 높아졌다. 
유의미한 숫자로 바꿔보자.

In [ ]:
data['Fare_cat']=0
data.loc[data['Fare']<=7.91,'Fare_cat']=0
data.loc[(data['Fare']>7.91)&(data['Fare']<=14.454),'Fare_cat']=1
data.loc[(data['Fare']>14.454)&(data['Fare']<=31),'Fare_cat']=2
data.loc[(data['Fare']>31)&(data['Fare']<=513),'Fare_cat']=3

In [ ]:
sns.factorplot('Fare_cat','Survived',data=data,hue='Sex')
plt.show()

임금이 높을 수록 살아남은 확률이 높다는 것을 입증했다. 

나중에, Modeling을 할 때, 성별과 관련시킨다면 시너지를 발휘할 수 있다. 

# Converting String Values into Numeric

In [ ]:
data['Sex'].replace(['male','female'],[0,1],inplace=True)
data['Embarked'].replace(['S','C','Q'],[0,1,2],inplace=True)
data['Initial'].replace(['Mr','Mrs','Miss','Master','Other'],[0,1,2,3,4],inplace=True)

### Dropping UnNeeded Features

Name : 철수, 영희 이름이 죽음에 연관이 있지 않다.

Age : Age_band할 때 사용했기 때문에, 더이상 필요없다.

Ticket : 무작위 문자열이라 필요없다.

Fare : Fare_cat할 때 사용했기 때문에, 더이상 필요없다.

Cabin : 널값들이 많고, 많은 사람들이 2개이상 이용했다. 그래서 객관적인 지표로 쓰기에는 무리가 있다.

Fare_Range : Fare_cat 특성을 만들었기 때문에, 필요없다.

Passengerld : 카테고리화 할 수 없다.


In [ ]:
data.drop(['Name','Age','Ticket','Fare','Cabin','Fare_Range','PassengerId'],axis=1,inplace=True)
sns.heatmap(data.corr(),annot=True,cmap='RdYlGn',linewidths=0.2,annot_kws={'size':20})
fig=plt.gcf()
fig.set_size_inches(18,15)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.show()

# Part3:Predictive Modeling

위와 같은 작업을 했지만, test_case를 넣었을 때, 그들이 살았을지 죽었을지 정확하게 판단할 수 없다. 이를 더 정확하게 하기 위해서는 알고리즘을 이용해야 한다.



1) Logistic Regression

2) Support Vector Machines(Linear and radial)

3) Random Forest

4) K-Nearest Neighbours

5) Naive Bayes

6) Decision Tree

7) Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split #train_test_split오류, pip install sklearn

In [ ]:
train,test=train_test_split(data,test_size=0.3,random_state=0,stratify=data['Survived'])
train_X=train[train.columns[1:]]
train_Y=train[train.columns[:1]]
test_X=test[test.columns[1:]]
test_Y=test[test.columns[:1]]
X=data[data.columns[1:]]
Y=data['Survived']

# Radial Support Vectop Machines(rbf-SVM)

In [ ]:
model=svm.SVC(kernel='rbf',C=1,gamma=0.1)
model.fit(train_X,train_Y)
prediction1=model.predict(test_X)
print('Accuracy for rbf SVM is ',metrics.accuracy_score(prediction1,test_Y))

# Linear Support Vector Machine(linear-SVM)

In [ ]:
model=svm.SVC(kernel='linear',C=0.1,gamma=0.1)
model.fit(train_X,train_Y)
prediction2=model.predict(test_X)
print('Accuracy for linear SVM is',metrics.accuracy_score(prediction2,test_Y))

# Logistic Regression

In [ ]:
model = LogisticRegression()
model.fit(train_X,train_Y)
prediction3=model.predict(test_X)
print('The accuracy of the Logistic Regression is',metrics.accuracy_score(prediction3,test_Y))

# Decision Tree

In [ ]:
model=DecisionTreeClassifier()
model.fit(train_X,train_Y)
prediction4=model.predict(test_X)
print('The accuracy of the Decision Tree is',metrics.accuracy_score(prediction4,test_Y))

# K-Nearest Neighbours(KNN)

In [ ]:
model=KNeighborsClassifier() 
model.fit(train_X,train_Y)
prediction5=model.predict(test_X)
print('The accuracy of the KNN is',metrics.accuracy_score(prediction5,test_Y))

값을 정확하게 하기위해, 이웃들을 5명으로 설정하겠다.

In [ ]:
a_index=list(range(1,11))
a=pd.Series()
x=[0,1,2,3,4,5,6,7,8,9,10]
for i in list(range(1,11)):
    model=KNeighborsClassifier(n_neighbors=i) 
    model.fit(train_X,train_Y)
    prediction=model.predict(test_X)
    a=a.append(pd.Series(metrics.accuracy_score(prediction,test_Y)))
plt.plot(a_index, a)
plt.xticks(x)
fig=plt.gcf()
fig.set_size_inches(12,6)
plt.show()
print('Accuracies for different values of n are:',a.values,'with the max value as ',a.values.max())

# Gaussian Naive Bayes

In [ ]:
model=GaussianNB()
model.fit(train_X,train_Y)
prediction6=model.predict(test_X)
print('The accuracy of the NaiveBayes is',metrics.accuracy_score(prediction6,test_Y))

# Random Forests

In [ ]:
model=RandomForestClassifier(n_estimators=100)
model.fit(train_X,train_Y)
prediction7=model.predict(test_X)
print('The accuracy of the Random Forests is',metrics.accuracy_score(prediction7,test_Y))

알고르즘 확률은 90%이상은 넘어가야 정확하다고 판단한다. 하지만, 다른 케이스를 넣을 때 마다, 확률이 변동될 수가 있다. 

이를 해결하기 위해서, 여러번의 테스트가 필요하다. 다양한 케이스들을 넣어 시도를 하고, 정확성을 높이자.

# Cross Validation

In [ ]:
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

In [ ]:
kfold = KFold(n_splits=10, shuffle=True, random_state=22) #랜덤의 여지가 없는데 랜덤하게 하라고 명령한 것. so 'shuffle=True'를 삽입
xyz=[]
accuracy=[]
std=[]
classifiers=['Linear Svm','Radial Svm','Logistic Regression','KNN','Decision Tree','Naive Bayes','Random Forest']
models=[svm.SVC(kernel='linear'),svm.SVC(kernel='rbf'),LogisticRegression(),KNeighborsClassifier(n_neighbors=9),DecisionTreeClassifier(),GaussianNB(),RandomForestClassifier(n_estimators=100)]
for i in models:
    model = i
    cv_result = cross_val_score(model,X,Y, cv = kfold,scoring = "accuracy")
    cv_result=cv_result
    xyz.append(cv_result.mean())
    std.append(cv_result.std())
    accuracy.append(cv_result)
new_models_dataframe2=pd.DataFrame({'CV Mean':xyz,'Std':std},index=classifiers)       
new_models_dataframe2

In [ ]:
plt.subplots(figsize=(12,6))
box=pd.DataFrame(accuracy,index=[classifiers])
box.T.boxplot()

In [ ]:
new_models_dataframe2['CV Mean'].plot.barh(width=0.8)
plt.title('Average CV Mean Accuracy')
fig=plt.gcf()
fig.set_size_inches(8,5)
plt.show()

불균형으로 인해, 분류 정확성을 잘못 도출할 수 있다. 그래서 이러한 혼란을 막기 위해 'confusion matrix'를 사용할 것이다. 

이는, 어디서 모델이 잘못되었는가? 어떤 class에서 잘못되었는 가를 말해준다.

# Confusion Matrix
classifier로 만들어진 요소중에 잘 된 분류와 잘 되지 않은 분류의 수를 알려준다.

In [ ]:
f,ax=plt.subplots(3,3,figsize=(12,10))
y_pred = cross_val_predict(svm.SVC(kernel='rbf'),X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[0,0],annot=True,fmt='2.0f')
ax[0,0].set_title('Matrix for rbf-SVM')
y_pred = cross_val_predict(svm.SVC(kernel='linear'),X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[0,1],annot=True,fmt='2.0f')
ax[0,1].set_title('Matrix for Linear-SVM')
y_pred = cross_val_predict(KNeighborsClassifier(n_neighbors=9),X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[0,2],annot=True,fmt='2.0f')
ax[0,2].set_title('Matrix for KNN')
y_pred = cross_val_predict(RandomForestClassifier(n_estimators=100),X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[1,0],annot=True,fmt='2.0f')
ax[1,0].set_title('Matrix for Random-Forests')
y_pred = cross_val_predict(LogisticRegression(),X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[1,1],annot=True,fmt='2.0f')
ax[1,1].set_title('Matrix for Logistic Regression')
y_pred = cross_val_predict(DecisionTreeClassifier(),X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[1,2],annot=True,fmt='2.0f')
ax[1,2].set_title('Matrix for Decision Tree')
y_pred = cross_val_predict(GaussianNB(),X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[2,0],annot=True,fmt='2.0f')
ax[2,0].set_title('Matrix for Naive Bayes')
plt.subplots_adjust(hspace=0.2,wspace=0.2)
plt.show()

왼쪽은 정확한 예측을 보여준다. 반면에, 오른쪽은 잘못된 예그측을 보여준다.

SVM : 정확한 예측 : 491명은 죽고, 247명은 살았다. CV정확성은 (491+247)/891 = 82.8%
    : 잘못된 예측 : 58명은 살았지만 죽은 사람으로 분류, 95명은 죽었지만 산사람으로 분류

결론적으로는, SVM은 죽은 사람을 예측하는 데 정확성이 높았으며, NaiveBayes는 산 사람을 예측하는 데 정확성이 높았다.

# Hyper=parameters Tuning

Svm은 C 그리고 gamma라는 매개변수가 있다.
유사하지만 다른 매개변수를 'Hyper-parameters'라고 한다. 

우리는 이를 조정하고 변화시켜 기계를 학습시킨다면, 더 높은 modeling을 할 수 있다.

### svm

In [ ]:
from sklearn.model_selection import GridSearchCV
C=[0.05,0.1,0.2,0.3,0.25,0.4,0.5,0.6,0.7,0.8,0.9,1]
gamma=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
kernel=['rbf','linear']
hyper={'kernel':kernel,'C':C,'gamma':gamma}
gd=GridSearchCV(estimator=svm.SVC(),param_grid=hyper,verbose=True)
gd.fit(X,Y)
print(gd.best_score_)
print(gd.best_estimator_)

#### Random Forests

In [ ]:
n_estimators=range(100,1000,100)
hyper={'n_estimators':n_estimators}
gd=GridSearchCV(estimator=RandomForestClassifier(random_state=0),param_grid=hyper,verbose=True)
gd.fit(X,Y)
print(gd.best_score_)
print(gd.best_estimator_)

# Ensembling

정확성을 높이기 위한 작업이다. 
다양한 model을 합쳐서 하나의 model로 만드는 것이다.

방법
1. Voting Classifier
2. Bagging
3. Boosting

# Voting Classifier

결합된 예측을 합치는 데 가장 간단한 방법이다.
모든 submodel들의 예측을 기반한 결과의 평균을 준다.

그리고, submodel과 basemodel은 모든 다른 type이다

In [ ]:
from sklearn.ensemble import VotingClassifier
ensemble_lin_rbf=VotingClassifier(estimators=[('KNN',KNeighborsClassifier(n_neighbors=10)),
                                              ('RBF',svm.SVC(probability=True,kernel='rbf',C=0.5,gamma=0.1)),
                                              ('RFor',RandomForestClassifier(n_estimators=500,random_state=0)),
                                              ('LR',LogisticRegression(C=0.05)),
                                              ('DT',DecisionTreeClassifier(random_state=0)),
                                              ('NB',GaussianNB()),
                                              ('svm',svm.SVC(kernel='linear',probability=True))
                                             ], 
                       voting='soft').fit(train_X,train_Y)
print('The accuracy for ensembled model is:',ensemble_lin_rbf.score(test_X,test_Y))
cross=cross_val_score(ensemble_lin_rbf,X,Y, cv = 10,scoring = "accuracy")
print('The cross validated score is',cross.mean())

# Bagging

앙상블 학습. 배깅은 샘플을 여러 번 뽑아(Bootstrap) 각 모델을 학습시켜 결과물을 집계(Aggregration)하는 방법입니다.

최종은 투표 방식으로 한다. 전체 모델에서 예측한 값 중 가장 많은 값을 최종 예측값으로 선정한다는 것입니다.
6개의 결정 트리 모델이 있다고 합시다. 

4개는 A로 예측했고, 2개는 B로 예측했다면 투표에 의해 4개의 모델이 선택한 A를 최종 결과로 예측한다는 것입니다.

ex) sample1, sample2, sample3, sample4 등을 학습시킨 후 결과물을 본다.

#### Bagged KNN

배깅은 분산이 잘 나타나는 곳에서 잘 작동한다. 예로, Decision Tree 또는 Random Forests에서 작동한다. 우리는 KNN에서도 구동할 수 있다.

In [ ]:
from sklearn.ensemble import BaggingClassifier
model=BaggingClassifier(base_estimator=KNeighborsClassifier(n_neighbors=3),random_state=0,n_estimators=700)
model.fit(train_X,train_Y)
prediction=model.predict(test_X)
print('The accuracy for bagged KNN is:',metrics.accuracy_score(prediction,test_Y))
result=cross_val_score(model,X,Y,cv=10,scoring='accuracy')
print('The cross validated score for bagged KNN is:',result.mean())

#### Bagged DecisionTree

In [ ]:
model=BaggingClassifier(base_estimator=DecisionTreeClassifier(),random_state=0,n_estimators=100)
model.fit(train_X,train_Y)
prediction=model.predict(test_X)
print('The accuracy for bagged Decision Tree is:',metrics.accuracy_score(prediction,test_Y))
result=cross_val_score(model,X,Y,cv=10,scoring='accuracy')
print('The cross validated score for bagged Decision Tree is:',result.mean())

# Boosting
머신러닝 앙상블 기법 중 하나이다. 연속적으로 weak learner들을 여러 개 결합하여 예측 혹은 분류 성능을 높이는 알고리즘이다.

약한 분류기가 순차적으로 학습하며, 앞에서 학습한 분류기가 예측이 틀린 데이터에 대해 다음 분류기가 가중치를 인가해서 학습을 이어 진행하는 방식

#### AdaBoost(Adaptive Boosting)

Decesion Tree을 통해 분류 기준으로 분류 된다. 단, 이는 다른 알고리즘으로 대체가능하다.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ada=AdaBoostClassifier(n_estimators=200,random_state=0,learning_rate=0.1)
result=cross_val_score(ada,X,Y,cv=10,scoring='accuracy')
print('The cross validated score for AdaBoost is:',result.mean())

#### Stochastic Gradient Bossting
여기서도 Decision Tree로 weak learner들을 분류한다.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
grad=GradientBoostingClassifier(n_estimators=500,random_state=0,learning_rate=0.1)
result=cross_val_score(grad,X,Y,cv=10,scoring='accuracy')
print('The cross validated score for Gradient Boosting is:',result.mean())

#### XGBoost

In [ ]:
import xgboost as xg
xgboost=xg.XGBClassifier(n_estimators=900,learning_rate=0.1)
result=cross_val_score(xgboost,X,Y,cv=10,scoring='accuracy')
print('The cross validated score for XGBoost is:',result.mean())

AdaBoost일 때, 정확성이 가장 높았다. 이제 우리는 Hyper_Parameter Tuning을 통해 이 정확성을 더욱 높이겠다.

#### Hyper-Parameter Tuning for AdaBoos

In [ ]:
n_estimators=list(range(100,1100,100))
learn_rate=[0.05,0.1,0.2,0.3,0.25,0.4,0.5,0.6,0.7,0.8,0.9,1]
hyper={'n_estimators':n_estimators,'learning_rate':learn_rate}
gd=GridSearchCV(estimator=AdaBoostClassifier(),param_grid=hyper,verbose=True)
gd.fit(X,Y)
print(gd.best_score_)
print(gd.best_estimator_)

우리가 가질 수 있는 Adaboost의 가장 높은 정확성은 83.16%다. n_extimators = 이고, learning_rate= 이다.

#### Confusion Matrix for the Best Model

In [ ]:
ada=AdaBoostClassifier(n_estimators=200,random_state=0,learning_rate=0.05)
result=cross_val_predict(ada,X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,result),cmap='winter',annot=True,fmt='2.0f')
plt.show()

# Feature Importance

In [ ]:
f,ax=plt.subplots(2,2,figsize=(15,12))
model=RandomForestClassifier(n_estimators=500,random_state=0)
model.fit(X,Y)
pd.Series(model.feature_importances_,X.columns).sort_values(ascending=True).plot.barh(width=0.8,ax=ax[0,0])
ax[0,0].set_title('Feature Importance in Random Forests')
model=AdaBoostClassifier(n_estimators=200,learning_rate=0.05,random_state=0)
model.fit(X,Y)
pd.Series(model.feature_importances_,X.columns).sort_values(ascending=True).plot.barh(width=0.8,ax=ax[0,1],color='#ddff11')
ax[0,1].set_title('Feature Importance in AdaBoost')
model=GradientBoostingClassifier(n_estimators=500,learning_rate=0.1,random_state=0)
model.fit(X,Y)
pd.Series(model.feature_importances_,X.columns).sort_values(ascending=True).plot.barh(width=0.8,ax=ax[1,0],cmap='RdYlGn_r')
ax[1,0].set_title('Feature Importance in Gradient Boosting')
model=xg.XGBClassifier(n_estimators=900,learning_rate=0.1)
model.fit(X,Y)
pd.Series(model.feature_importances_,X.columns).sort_values(ascending=True).plot.barh(width=0.8,ax=ax[1,1],color='#FD0F00')
ax[1,1].set_title('Feature Importance in XgBoost')
plt.show()

RandomForests와 AdaBoost와 같은 다양한 중요한 특징을 볼 수 있다.

관측

1. Initial, Fare_cat, Pclass, Family_Size 특성이 주요했다.

2. 성별은 Decesion Tree에서만 두각을 들어내고 다른 곳에서는 주요했지 않았지만, 클래스와 연관시켰을 땐, 주요했다.

3. Pclass 및 Fare_cat은 혼자, Parch 및 Sibsp와 함께 승객 및 Family_Size의 상태를 나타난다.